<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/script_VAEGAN_voxelsize1_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mount google drive
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#right click shared folder IRCMS_GAN_collaborative_database and "Add shortcut to Drive" to My drive
%cd drive/My Drive/IRCMS_GAN_collaborative_database/Experiments/colab-treegan/

#record paths to resources
data_path = "../../Research/Peter/Tree_3D_models_obj/obj_files/"
run_path = "./"

# !ls

/content/drive/.shortcut-targets-by-id/1ylB2p6N0qQ-G4OsBuwcZ9C0tsqVu9ww4/IRCMS_GAN_collaborative_database/Experiments/colab-treegan


In [3]:
!pip install pytorch-lightning
!pip install trimesh
!pip install wandb -q
!apt install -y xvfb
!pip install trimesh xvfbwrapper
output.clear()

#add libraries, and login to wandb

In [4]:
import io
import os
import trimesh
import numpy as np

import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb

from xvfbwrapper import Xvfb

In [5]:
!wandb login
output.clear()

In [6]:
#id None to start a new run. For resuming run, put the id of the run below
id = None 
resume = False
if id is None:
    id = wandb.util.generate_id()
else:
    resume = True

run = wandb.init(project="tree-gan", id=id, resume="allow", dir=run_path)
print("run id:" + str(wandb.run.id))
print("run name: " + str(wandb.run.name))
wandb.watch_called = False
wandb.run.save_code = True

run id:3nerxvor
run name: 3nerxvor


In [7]:
#keep track of hyperparams
config = wandb.config
config.dataset = "dataset-tree:3"

config.batch_size = 8
config.array_size = 32

config.z_size = 128
config.gen_num_layer_unit = [32, 32, 32, 32]
config.dis_num_layer_unit = [16, 16, 16, 16]
config.leakyReLU = False
config.balance_voxel_in_space = False

config.epochs = 10
config.vae_lr = 0.001
config.vae_encoder_layer = 1
config.vae_decoder_layer = 2
config.d_lr = 0.0003         
config.d_layer = 1
config.vae_recon_loss_factor = 1
config.seed = 1234
config.log_image_interval = 5
config.log_mesh_interval = 50
config.data_augmentation = True
config.num_augment_data = 4

config.vae_opt = "Adam"
config.dis_opt = "Adam"


#dataset

In [8]:
#our package

#clone then install
# !git clone https://github.com/buganart/BUGAN repo
# !pip install -e ./repo/
# import site
# site.main()

#directly install using pip
# !pip install git+https://github.com/buganart/BUGAN.git#egg=bugan
# output.clear()

In [9]:
# from bugan.functions import *
# from bugan.models import VAEGAN, VAE, Discriminator, Generator

from functionsPL import *
from modelsPL import VAEGAN, VAE, Discriminator, Generator

run.tags.append("VAEGAN")
run.group = "VAEGAN"

In [10]:
np.random.seed(config.seed)
dataModule = DataModule(config, run)

#functions for pytorch network

#train

In [11]:
#load model
vaegan = VAEGAN(config).to(device)

# if resume:
#     vaegan = load_model(vaegan)
summary(vaegan, (1, config.array_size, config.array_size, config.array_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 16, 32, 32, 32]             448
       BatchNorm3d-2       [-1, 16, 32, 32, 32]              32
              ReLU-3       [-1, 16, 32, 32, 32]               0
              ReLU-4       [-1, 16, 32, 32, 32]               0
         MaxPool3d-5       [-1, 16, 16, 16, 16]               0
            Conv3d-6       [-1, 16, 16, 16, 16]           6,928
       BatchNorm3d-7       [-1, 16, 16, 16, 16]              32
              ReLU-8       [-1, 16, 16, 16, 16]               0
              ReLU-9       [-1, 16, 16, 16, 16]               0
        MaxPool3d-10          [-1, 16, 8, 8, 8]               0
           Conv3d-11          [-1, 16, 8, 8, 8]           6,928
      BatchNorm3d-12          [-1, 16, 8, 8, 8]              32
             ReLU-13          [-1, 16, 8, 8, 8]               0
             ReLU-14          [-1, 16, 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [12]:
#set seed
torch.manual_seed(config.seed)
torch.autograd.set_detect_anomaly(True)

#render setup
vdisplay = Xvfb()
vdisplay.start()

trainer = pl.Trainer()
trainer.fit(vaegan, dataModule)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name          | Type          | Params
------------------------------------------------
0 | vae           | VAE           | 453 K 
1 | discriminator | Discriminator | 152 K 
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on 

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:3121: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)
Saving latest checkpoint..


1